In [1]:
import tellurium as te
import pandas as pd
import numpy as np

import cobra
import os
os.getcwd()


from src.no_BayesianInference import BMCA, util

In [2]:
os.getcwd()

'c:\\Users\\user\\Documents\\research\\BMCA-pipeline'

Generating data for running through the pipeline.

Running BMCA on BioModel 64

In [3]:
# load the Teusink model
ant_file = 'data/interim/Antimony/BIOMD0000000064_e.ant'
data_file = 'data/interim/generated_data/BIOMD0000000064_e_50.csv'
data = pd.read_csv(data_file)

Calculating the steady state concentrations of the Teusink model when EtOH concentration is halved. 

In [4]:
s  = te.loada(ant_file)
s.ETOH = s.ETOH/2
s.steadyState()
true_ss_values = pd.DataFrame(s.getFloatingSpeciesConcentrations(), index=s.getFloatingSpeciesIds(), columns=['true'])

# s.getScaledFluxControlCoefficientMatrix()

Predicting the steady state concentration values 

In [8]:
BIOMD64 = BMCA.BMCA(ant_file, data_file, desired_product='ETOH')

# pd.DataFrame(pipeline.Ex, columns=s.getFloatingSpeciesIds(), index=s.getReactionIds())
# pd.DataFrame(pipeline.Ey, columns=s.getBoundarySpeciesIds(), index=s.getReactionIds())

chi, v_hat = BIOMD64.calculate_steady_state(BIOMD64.Ex, BIOMD64.Ey)
x_ss = np.exp(chi.to_numpy())

estimated_ss = pd.DataFrame(data=x_ss, index=s.getFloatingSpeciesIds(), columns=['estimated'])

c:\Users\user\Documents\research\BMCA-pipeline\src\no_BayesianInference\BMCA.py:143: FutureWarning:

Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.



In [11]:
comparison = pd.concat([true_ss_values, estimated_ss], axis = 1)
comparison['error (%)'] = (comparison['true']-comparison['estimated'])*100/comparison['true']
comparison.sort_index()

,true,estimated,error (%)
ACE,0.136863,0.039947,70.812336
BPG,0.000510,0.212927,-41629.273918
F16P,0.652591,0.039749,93.909018
F6P,0.156413,0.083391,46.685673
G6P,1.283076,0.001627,99.873183
GLCi,0.086895,0.133252,-53.348152
NAD,1.559295,0.000548,99.964846
NADH,0.030705,1.359586,-4327.917403
P,6.769148,0.202326,97.011062
P2G,0.054551,0.013524,75.207967
